In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
import datetime as dt
from sklearn.metrics import classification_report

In [2]:
#load data
file = 'spambase/spambase.data'
dataframe=pd.read_csv(file,header=None)
X = dataframe.iloc[:,:57]
y = dataframe.iloc[:, 57]

#stratified fold
skfold = StratifiedKFold(n_splits=10,shuffle=False)
#shuffling gives a good accuracy
#skfold = StratifiedKFold(n_splits=10,shuffle=True, random_state=300)

In [3]:
#SVM
#gamma=auto is used to avoid future warnings
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

scoring = ['accuracy', 'f1']
model_skfold = svm.SVC(gamma='auto')
results_skfold = model_selection.cross_validate(model_skfold, X, y, cv=skfold, scoring=scoring)
SVM_DF=pd.DataFrame((results_skfold))
SVM_DF=np.around(SVM_DF,4)
#SVM_DF

In [4]:
#MLP
from sklearn.neural_network import MLPClassifier

model_skfold1 = MLPClassifier()
results_skfold1 = model_selection.cross_validate(model_skfold1, X, y, cv=skfold, scoring=scoring)
MLP_DF=pd.DataFrame((results_skfold1))
MLP_DF=np.around(MLP_DF,4)
#MLP_DF

In [5]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

model_skfold2 = KNeighborsClassifier(n_neighbors =1)
results_skfold2 = model_selection.cross_validate(model_skfold2, X, y, cv=skfold, scoring=scoring)
KNN_DF=pd.DataFrame((results_skfold2))
KNN_DF=np.around(KNN_DF,4)
#KNN_DF

In [6]:
#Accuracy cross validation
ACC_DF=pd.DataFrame()
ACC_DF['SVM']=SVM_DF['test_accuracy']
ACC_DF['MLP']=MLP_DF['test_accuracy']
ACC_DF['KNN']=KNN_DF['test_accuracy']

ACC_avg=pd.DataFrame()
ACC_sd=pd.DataFrame()

#Average of Accuracy
ACC_avg = pd.DataFrame({"SVM":[round(ACC_DF['SVM'].mean(),4)], 
                  "MLP":[round(ACC_DF['MLP'].mean(),4)],
                  "KNN":[round(ACC_DF['KNN'].mean(),4)]})

#Standard Deviation of Accuracy
ACC_sd = pd.DataFrame({"SVM":[round(ACC_DF['SVM'].std(),4)], 
                  "MLP":[round(ACC_DF['MLP'].std(),4)],
                  "KNN":[round(ACC_DF['KNN'].std(),4)]})

#Table 12.4 for Accuarcy
ACC_DF_CV=pd.DataFrame()
ACC_DF_CV = ACC_DF.append(ACC_avg,ignore_index=True)
ACC_DF_CV = ACC_DF_CV.append(ACC_sd,ignore_index=True)
ACC_DF_CV = ACC_DF_CV.rename(index={10: 'avg'})
ACC_DF_CV = ACC_DF_CV.rename(index={11: 'std'})
ACC_DF_CV

,SVM,MLP,KNN
0,0.7787,0.9132,0.7505
1,0.8000,0.9130,0.8000
2,0.8152,0.9348,0.7913
3,0.8457,0.9543,0.8022
4,0.8152,0.9543,0.8348
5,0.8565,0.9435,0.8174
6,0.8348,0.9674,0.8174
7,0.8457,0.9500,0.8217
8,0.7500,0.8543,0.7283
9,0.8022,0.8609,0.7217


In [7]:
#Accuracy ranking
ACC_RANK = pd.DataFrame()
ACC_RANK = ACC_DF.rank(axis=1,ascending=False)
ACC_RANK = ACC_RANK.astype(int)
#print(ACC_RANK)

#Table 12.8 for Accuracy
table2=pd.DataFrame()
table2['SVM']= ACC_DF['SVM'].map(str) + '(' + ACC_RANK['SVM'].map(str)+')'
table2['MLP']= ACC_DF['MLP'].map(str) + '(' + ACC_RANK['MLP'].map(str)+')'
table2['KNN']= ACC_DF['KNN'].map(str) + '(' + ACC_RANK['KNN'].map(str)+')'

#Average Rank
td2_avg=pd.DataFrame({"SVM":[round(ACC_RANK['SVM'].mean(),4)], 
                  "MLP":[round(ACC_RANK['MLP'].mean(),4)],
                  "KNN":[round(ACC_RANK['KNN'].mean(),4)]})

#Final Table including average accuracy, standarad deviation and ranking
FT_Accuracy = table2.append(ACC_avg,ignore_index=True)
FT_Accuracy = FT_Accuracy.append(ACC_sd,ignore_index=True)
FT_Accuracy = FT_Accuracy.append(td2_avg,ignore_index=True)
FT_Accuracy = FT_Accuracy.rename(index={10: 'avg'})
FT_Accuracy = FT_Accuracy.rename(index={11: 'std'})
FT_Accuracy = FT_Accuracy.rename(index={12: 'avg rank'})
FT_Accuracy

,SVM,MLP,KNN
0,0.7787(2),0.9132(1),0.7505(3)
1,0.8(2),0.913(1),0.8(2)
2,0.8152(2),0.9348(1),0.7913(3)
3,0.8457(2),0.9543(1),0.8022(3)
4,0.8152(3),0.9543(1),0.8348(2)
5,0.8565(2),0.9435(1),0.8174(3)
6,0.8348(2),0.9674(1),0.8174(3)
7,0.8457(2),0.95(1),0.8217(3)
8,0.75(2),0.8543(1),0.7283(3)
9,0.8022(2),0.8609(1),0.7217(3)


In [8]:
#Friedman Test for Accuracy
from scipy.stats import friedmanchisquare

print('H0: All algroithms perform equally according to accuracy')
print('H1: Not all algorithms perform equally according to accuracy')
rank1=ACC_RANK["SVM"]
rank2=ACC_RANK["MLP"]
rank3=ACC_RANK["KNN"]
stat, p = friedmanchisquare(rank1,rank2,rank3)
print('Friedman Test Statistic:',round(stat,4))
print('p value:',round(p,5))

alpha = 0.05
print('Alpha:',alpha)
if p > alpha:
    print('p value > alpha')
    print('fail to reject H0')

else:
    print('p value < alpha')
    print('Reject H0')
    print('Not all algorithms perform equally according to accuracy')

H0: All algroithms perform equally according to accuarcy
H1: Not all algorithms perform equally according to accuarcy
Friedman Test Statistic: 17.8974
p value: 0.00013
Alpha: 0.05
p value < alpha
Reject H0
Not all algorithms perform equally according to accuarcy


In [10]:
#Training Time cross validation
Time_DF=pd.DataFrame()
Time_DF['SVM']=SVM_DF['fit_time']
Time_DF['MLP']=MLP_DF['fit_time']
Time_DF['KNN']=KNN_DF['fit_time']

Time_avg=pd.DataFrame()
Time_sd=pd.DataFrame()

#Average of Training Time
Time_avg = pd.DataFrame({"SVM":[round(Time_DF['SVM'].mean(),4)], 
                  "MLP":[round(Time_DF['MLP'].mean(),4)],
                  "KNN":[round(Time_DF['KNN'].mean(),4)]})

#Standard Deviation of Training Time
Time_sd = pd.DataFrame({"SVM":[round(Time_DF['SVM'].std(),4)], 
                  "MLP":[round(Time_DF['MLP'].std(),4)],
                  "KNN":[round(Time_DF['KNN'].std(),4)]})

#Table 12.4 for Training Time
Time_DF_CV=pd.DataFrame()
Time_DF_CV = Time_DF.append(Time_avg,ignore_index=True)
Time_DF_CV = Time_DF_CV.append(Time_sd,ignore_index=True)
Time_DF_CV = Time_DF_CV.rename(index={10: 'avg'})
Time_DF_CV = Time_DF_CV.rename(index={11: 'std'})
Time_DF_CV

,SVM,MLP,KNN
0,1.0886,1.5602,0.0166
1,1.0023,1.8857,0.0210
2,1.0012,1.8066,0.0162
3,1.0028,1.0498,0.0166
4,1.0247,1.7746,0.0158
5,1.0080,1.5601,0.0149
6,1.0304,1.8891,0.0139
7,0.9882,1.1323,0.0145
8,0.9740,1.2912,0.0139
9,0.9911,1.5724,0.0158


In [11]:
#Training Time ranking
Time_RANK = pd.DataFrame()
Time_RANK = Time_DF.rank(axis=1,ascending=True)
Time_RANK = Time_RANK.astype(int)
#print(Time_RANK)

#Table 12.8 for Training Time
table2=pd.DataFrame()
table2['SVM']= Time_DF['SVM'].map(str) + '(' + Time_RANK['SVM'].map(str)+')'
table2['MLP']= Time_DF['MLP'].map(str) + '(' + Time_RANK['MLP'].map(str)+')'
table2['KNN']= Time_DF['KNN'].map(str) + '(' + Time_RANK['KNN'].map(str)+')'

td2_avg=pd.DataFrame({"SVM":[round(Time_RANK['SVM'].mean(),4)], 
                  "MLP":[round(Time_RANK['MLP'].mean(),4)],
                  "KNN":[round(Time_RANK['KNN'].mean(),4)]})

#Final Table including average training time, standarad deviation and ranking
FT_Time = table2.append(Time_avg,ignore_index=True)
FT_Time = FT_Time.append(Time_sd,ignore_index=True)
FT_Time = FT_Time.append(td2_avg,ignore_index=True)
FT_Time = FT_Time.rename(index={10: 'avg'})
FT_Time = FT_Time.rename(index={11: 'std'})
FT_Time = FT_Time.rename(index={12: 'avg rank'})
FT_Time

,SVM,MLP,KNN
0,1.0886(2),1.5602(3),0.0166(1)
1,1.0023(2),1.8857(3),0.021(1)
2,1.0012(2),1.8066(3),0.0162(1)
3,1.0028(2),1.0498(3),0.0166(1)
4,1.0247(2),1.7746(3),0.0158(1)
5,1.008(2),1.5601(3),0.0149(1)
6,1.0304(2),1.8891(3),0.0139(1)
7,0.9882(2),1.1323(3),0.0145(1)
8,0.974(2),1.2912(3),0.0139(1)
9,0.9911(2),1.5724(3),0.0158(1)


In [12]:
#Friedman Test for Training Time
from scipy.stats import friedmanchisquare

print('H0: All algroithms perform equally according to Training Time')
print('H1: Not all algorithms perform equally according to Training Time')
rank1=Time_RANK["SVM"]
rank2=Time_RANK["MLP"]
rank3=Time_RANK["KNN"]
stat, p = friedmanchisquare(rank1,rank2,rank3)
print('Friedman Test Statistic:',round(stat,4))
print('p value:',round(p,5))

alpha = 0.05
print('Alpha:',alpha)
if p > alpha:
    print('p value > alpha')
    print('fail to reject H0')

else:
    print('p value < alpha')
    print('Reject H0')
    print('Not all algorithms perform equally according to Training Time')

H0: All algroithms perform equally according to Training Time
H1: Not all algorithms perform equally according to Training Time
Friedman Test Statistic: 20.0
p value: 5e-05
Alpha: 0.05
p value < alpha
Reject H0
Not all algorithms perform equally according to Training Time


In [13]:
#FScore cross validation
Fscore_DF=pd.DataFrame()
Fscore_DF['SVM']=SVM_DF['test_f1']
Fscore_DF['MLP']=MLP_DF['test_f1']
Fscore_DF['KNN']=KNN_DF['test_f1']

Fscore_avg=pd.DataFrame()
Fscore_sd=pd.DataFrame()

#Average of Fscore
Fscore_avg = pd.DataFrame({"SVM":[round(Fscore_DF['SVM'].mean(),4)], 
                  "MLP":[round(Fscore_DF['MLP'].mean(),4)],
                  "KNN":[round(Fscore_DF['KNN'].mean(),4)]})

#Standard Deviation of Fscore
Fscore_sd = pd.DataFrame({"SVM":[round(Fscore_DF['SVM'].std(),4)], 
                  "MLP":[round(Fscore_DF['MLP'].std(),4)],
                  "KNN":[round(Fscore_DF['KNN'].std(),4)]})

#Table 12.4 for Fscore
Fscore_DF_CV=pd.DataFrame()
Fscore_DF_CV = Fscore_DF.append(Fscore_avg,ignore_index=True)
Fscore_DF_CV = Fscore_DF_CV.append(Fscore_sd,ignore_index=True)
Fscore_DF_CV = Fscore_DF_CV.rename(index={10: 'avg'})
Fscore_DF_CV = Fscore_DF_CV.rename(index={11: 'std'})
Fscore_DF_CV

,SVM,MLP,KNN
0,0.7302,0.8925,0.6814
1,0.7473,0.8936,0.7356
2,0.7658,0.9167,0.7348
3,0.8022,0.9415,0.7437
4,0.7757,0.9405,0.7877
5,0.8290,0.9312,0.7778
6,0.7841,0.9570,0.7572
7,0.8097,0.9345,0.7796
8,0.7160,0.8312,0.6719
9,0.7612,0.8140,0.6614


In [14]:
#Fscore ranking
Fscore_RANK = pd.DataFrame()
Fscore_RANK = Fscore_DF.rank(axis=1,ascending=False)
Fscore_RANK = Fscore_RANK.astype(int)
#print(Fscore_RANK)

#Table 12.8 for Fscore
table2=pd.DataFrame()
table2['SVM']= Fscore_DF['SVM'].map(str) + '(' + Fscore_RANK['SVM'].map(str)+')'
table2['MLP']= Fscore_DF['MLP'].map(str) + '(' + Fscore_RANK['MLP'].map(str)+')'
table2['KNN']= Fscore_DF['KNN'].map(str) + '(' + Fscore_RANK['KNN'].map(str)+')'
td2_avg=pd.DataFrame({"SVM":[round(Fscore_RANK['SVM'].mean(),4)], 
                  "MLP":[round(Fscore_RANK['MLP'].mean(),4)],
                  "KNN":[round(Fscore_RANK['KNN'].mean(),4)]})

#Final Table including average fscore, standarad deviation and ranking
FT_Fscore = table2.append(Fscore_avg,ignore_index=True)
FT_Fscore = FT_Fscore.append(Fscore_sd,ignore_index=True)
FT_Fscore = FT_Fscore.append(td2_avg,ignore_index=True)
FT_Fscore = FT_Fscore.rename(index={10: 'avg'})
FT_Fscore = FT_Fscore.rename(index={11: 'std'})
FT_Fscore = FT_Fscore.rename(index={12: 'avg rank'})
FT_Fscore

,SVM,MLP,KNN
0,0.7302(2),0.8925(1),0.6814(3)
1,0.7473(2),0.8936(1),0.7356(3)
2,0.7658(2),0.9167(1),0.7348(3)
3,0.8022(2),0.9415(1),0.7437(3)
4,0.7757(3),0.9405(1),0.7877(2)
5,0.829(2),0.9312(1),0.7778(3)
6,0.7841(2),0.957(1),0.7572(3)
7,0.8097(2),0.9345(1),0.7796(3)
8,0.716(2),0.8312(1),0.6719(3)
9,0.7612(2),0.814(1),0.6614(3)


In [15]:
#Friedman Test for Fscore
from scipy.stats import friedmanchisquare

print('H0: All algroithms perform equally according to the Fscore')
print('H1: Not all algorithms perform equally according to the Fscore')
rank1=Fscore_RANK["SVM"]
rank2=Fscore_RANK["MLP"]
rank3=Fscore_RANK["KNN"]
stat, p = friedmanchisquare(rank1,rank2,rank3)
print('Friedman Test Statistic:',round(stat,4))
print('p value:',round(p,5))

alpha = 0.05
print('Alpha:',alpha)
if p > alpha:
    print('p value > alpha')
    print('fail to reject H0')

else:
    print('p value < alpha')
    print('Reject H0')
    print('Not all algorithms perform equally according to the Fscore')

H0: All algroithms perform equally according to the Fscore
H1: Not all algorithms perform equally according to the Fscore
Friedman Test Statistic: 18.2
p value: 0.00011
Alpha: 0.05
p value < alpha
Reject H0
Not all algorithms perform equally according to the Fscore
